In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5
from pycromanager import Core, Acquisition
from pathlib import Path
import json
import multiprocessing as mp

In [2]:
events_file_path = Path('F:\live_analysis_test\events.json')
with open(events_file_path, 'r') as f:
    events = json.load(f)

In [3]:
events

[{'axes': {'position': 1, 'preset': 'phase_fast', 'time': 0},
  'config_group': ['imaging', 'phase_fast'],
  'exposure': 30,
  'extra_tags': {'is_dummy': False, 'position': 1},
  'min_start_time': 0,
  'x': 1262.4,
  'y': -1906.1000000000001,
  'z': 4105.0},
 {'axes': {'position': 1, 'preset': 'venus', 'time': 0},
  'config_group': ['imaging', 'venus'],
  'exposure': 50,
  'extra_tags': {'is_dummy': False, 'position': 1},
  'min_start_time': 0,
  'x': 1262.4,
  'y': -1906.1000000000001,
  'z': 4105.0},
 {'axes': {'position': 2, 'preset': 'phase_fast', 'time': 0},
  'config_group': ['imaging', 'phase_fast'],
  'exposure': 30,
  'extra_tags': {'is_dummy': False, 'position': 2},
  'min_start_time': 0,
  'x': 1127.5214285714287,
  'y': -1906.6857142857143,
  'z': 4105.0},
 {'axes': {'position': 2, 'preset': 'venus', 'time': 0},
  'config_group': ['imaging', 'venus'],
  'exposure': 50,
  'extra_tags': {'is_dummy': False, 'position': 2},
  'min_start_time': 0,
  'x': 1127.5214285714287,
  'y

In [4]:
class Events:
    def __init__(self, events):
        self.i = 0
        self.events = events
        self.max = len(self.events)

    def __next__(self):
        if self.i < self.max:
            event = self.events[self.i]
            self.i += 1
            return event
        else:
            return None

In [5]:
all_images = []

In [6]:

def acquire_images(events):
    e = Events(events)
    def put_images_in_queue(image, metadata, event_queue):
        print(metadata['Axes'], '----->', image.shape)

        if not hasattr(put_images_in_queue, 'datapoint'):
            put_images_in_queue.datapoint = {}

        img_type = metadata['Axes']['preset']
        if img_type == 'phase_fast':
            key = 'phase'
        elif img_type == 'venus':
            key = 'fluor'
            
        put_images_in_queue.datapoint[key] = image
        put_images_in_queue.datapoint['position'] = metadata['Axes']['position']
        put_images_in_queue.datapoint['timepoint'] = metadata['Axes']['time']

        #print(put_images_in_queue.datapoint.keys())

        if 'phase' in put_images_in_queue.datapoint.keys() and 'fluor' in put_images_in_queue.datapoint.keys():
            all_images.append(put_images_in_queue.datapoint)
            #print('Put a datapoint in the queue')
            put_images_in_queue.datapoint = {}
        

        next_event = next(e)
        #print(f"Next event: {next_event}")
        event_queue.put(next_event)
        return 
    acq = Acquisition(name='test', image_process_fn=put_images_in_queue, show_display=False)
    acq.acquire(next(e))
    acq.await_completion()

    print(f"Len of all images: {len(all_images)}")

In [7]:
acquire_images(events)

{'position': 1, 'preset': 'phase_fast', 'time': 0} -----> (1380, 3020)
{'position': 1, 'preset': 'venus', 'time': 0} -----> (1380, 3020)
{'position': 2, 'preset': 'phase_fast', 'time': 0} -----> (1380, 3020)
{'position': 2, 'preset': 'venus', 'time': 0} -----> (1380, 3020)
{'position': 3, 'preset': 'phase_fast', 'time': 0} -----> (1380, 3020)
{'position': 3, 'preset': 'venus', 'time': 0} -----> (1380, 3020)
{'position': 4, 'preset': 'phase_fast', 'time': 0} -----> (1380, 3020)
{'position': 4, 'preset': 'venus', 'time': 0} -----> (1380, 3020)
{'position': 5, 'preset': 'phase_fast', 'time': 0} -----> (1380, 3020)
{'position': 5, 'preset': 'venus', 'time': 0} -----> (1380, 3020)
{'position': 6, 'preset': 'phase_fast', 'time': 0} -----> (1380, 3020)
{'position': 6, 'preset': 'venus', 'time': 0} -----> (1380, 3020)
{'position': 7, 'preset': 'phase_fast', 'time': 0} -----> (1380, 3020)
{'position': 7, 'preset': 'venus', 'time': 0} -----> (1380, 3020)
{'position': 8, 'preset': 'phase_fast', '

In [8]:
len(all_images)

30

In [9]:
all_images[0]

{'phase': array([[31852, 30539, 30478, ..., 31922, 31218, 30909],
        [31191, 31349, 31461, ..., 31520, 30772, 32096],
        [31172, 31310, 30719, ..., 31959, 32438, 31612],
        ...,
        [17459, 17778, 16756, ..., 16349, 16194, 15832],
        [16473, 16883, 16989, ..., 15349, 16192, 15407],
        [17343, 17417, 17340, ..., 15793, 15489, 15907]], dtype=uint16),
 'position': 1,
 'timepoint': 0,
 'fluor': array([[204, 222, 212, ..., 222, 208, 198],
        [183, 200, 210, ..., 212, 208, 198],
        [206, 218, 230, ..., 200, 213, 211],
        ...,
        [216, 207, 198, ..., 198, 197, 198],
        [208, 218, 200, ..., 221, 209, 214],
        [210, 257, 219, ..., 205, 219, 209]], dtype=uint16)}

In [10]:
i = 3
fig, ax = plt.subplots(nrows=2, ncols=1)
ax[0].imshow(all_images[i]['phase'], cmap='gray')
ax[1].imshow(all_images[i]['fluor'], cmap='gray')
plt.show()

In [11]:
import numpy as np
from rtseg.utils.param_io import load_params
from pathlib import Path
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread
from rtseg.forkplot import compute_forkplot_stats
import matplotlib.patches as patches
from rtseg.dotdetect import compute_dot_coordinates

In [12]:
from rtseg.segmentation import LiveNet, get_live_model, live_segment

C:\Users\elflab\Documents\Praneeth\rtclient\.venv\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: '2.0.2' (you have '2.0.0'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [13]:
params_path = Path("C:\\Users\\elflab\\Documents\\Praneeth\\rtseg\\rtseg\\resources\\reference_params\\reference_microscope.yaml")
params = load_params(params_path, ref_type='expt')

In [14]:
live_net = get_live_model(params)

In [119]:
i = 0
datapoint = all_images[i]

In [120]:
seg_result = live_segment(datapoint, live_net, params)

In [121]:
seg_result.keys()

dict_keys(['phase', 'fluor', 'position', 'timepoint', 'barcode_locations', 'seg_mask', 'trap_locations_list', 'num_traps', 'error'])

In [122]:
fig, ax = plt.subplots(nrows=2, ncols=1)
ax[0].imshow(seg_result['seg_mask'], cmap='viridis')
ax[1].imshow(seg_result['phase'], cmap='gray')
for row in seg_result['barcode_locations']:
    rect = patches.Rectangle((row[0], row[1]), row[2] - row[0], row[3] - row[1], linewidth=1, edgecolor='r', facecolor='none')
    ax[1].add_patch(rect)
plt.suptitle(f"Num traps: {seg_result['num_traps']}")
plt.show()

In [29]:
params.Dots.noise_threshold = 3.2
params.Dots.wavelet_plane_no = 2

In [32]:
for i in range(15, 30):
    datapoint = all_images[i]
    if i in range(15, 30):
        datapoint['phase'] = np.flipud(datapoint['phase'])
        datapoint['fluor'] = np.flipud(datapoint['fluor'])
    seg_result = live_segment(datapoint, live_net, params)
    dots= compute_dot_coordinates(seg_result['fluor'], seg_result['seg_mask'], params)
    fig, ax = plt.subplots(nrows=3, ncols=1)
    ax[0].imshow(seg_result['seg_mask'], cmap='viridis')
    ax[1].imshow(seg_result['phase'], cmap='gray')
    for row in seg_result['barcode_locations']:
        rect = patches.Rectangle((row[0], row[1]), row[2] - row[0], row[3] - row[1], linewidth=1, edgecolor='r', facecolor='none')
        ax[1].add_patch(rect)
    ax[2].imshow(seg_result['fluor'], cmap='gray')
    ax[2].plot(dots['raw_coords'][:, 1], dots['raw_coords'][:, 0], 'ro')
    plt.suptitle(f"Num traps: {seg_result['num_traps']}")
    plt.show()

In [25]:
i = 15
datapoint = all_images[i]
seg_result = live_segment(datapoint, live_net, params)
dots= compute_dot_coordinates(datapoint['fluor'], seg_result['seg_mask'], params)
fig, ax = plt.subplots(nrows=3, ncols=1)
ax[0].imshow(seg_result['seg_mask'], cmap='viridis')
ax[1].imshow(seg_result['phase'], cmap='gray')
for row in seg_result['barcode_locations']:
    rect = patches.Rectangle((row[0], row[1]), row[2] - row[0], row[3] - row[1], linewidth=1, edgecolor='r', facecolor='none')
    ax[1].add_patch(rect)
ax[2].imshow(seg_result['fluor'], cmap='gray')
ax[2].plot(dots['raw_coords'][:, 1], dots['raw_coords'][:, 0], 'ro')
plt.suptitle(f"Num traps: {seg_result['num_traps']}")
plt.show()

In [24]:
plt.figure()
plt.imshow(datapoint['fluor'], cmap='gray')
plt.show()